# Representación de las palabras

Como hemos visto en el tema de redes convolucionales, las imágenes se representan como la cantidad de rojo, verde y azul de cada pixel. Esa cantidad es un número que varía de entre 0 y 255. Es decir, para representar una imagen necesitamos hacerlo mediante números.

Con el lenguaje pasa igual, para poder procesarlo y realizar predicciones o generar texto, necesitamos poder representarlo mediante números. Vamos a ver varias formas de representar el lenguaje mediante números: `encoding ordinal`, `one-hot encoding` y `word embedding`

## Encoding ordinal

Esta es la manera más básica de representar numéricamente un lenguaje, y consiste en asignar un número a cada palabra, por ejemplo, podemos decir que gato lo representaremos con un 1, perro con un 2, mesa con un 3, ...

## One-hot encoding

Aunque el encoding ordinal nos resuelve el problema tiene varios problemas

 * En el ejemplo que hemos dado se puede asumir que mesa (3) equivale a gato (1) + perro (2), lo cual no tiene nada que ver. En el lenguaje existen relaciones entre las palabras, por ejemplo perro y perra tienen mucha relación, por lo que es necesario un sistema de codificación mejor
 * Importancia de las palabras. Las palabras van a entrar a redes neuronales, que como hemos visto se componene de capas, con unos pesos, mediante las cuales se realizan unas operaciones matemáticas. Por lo que puede llegar a pasar que la red le de más importancia a las palabras con un número mayor

Debido a esto, se pensó en una alternativa, el one-hot encodding. Aquí lo que se hace es crear vectores de tamaño N, donde cada palabra corresponderá a un vector de todo ceros, menos un uno en una posición determinada. En el ejemplo de antes gato correspondería al vector `[1 0 0 ... 0]`, perro al vector `[0 1 0 ... 0]`, mesa al vector `[0 0 1 ... 0]`, ...

Haciendo esta codificación arreglamos los dos problemas que hemos contados del encoding ordinal y además añadimos una ventaja, como las redes neuronales realizan operaciones matriciales, si a una matriz le multiplicas por un vector de todo ceros, menos un uno en una posición, lo que estás haciendo es que el resultado de la operación es obtener la columna o la fila correspondiente a la posición del 1

![one-hot encodding matrix multiplication](Imagenes/one_hot_encoding_matrix_multiplication.png)

Esto es muy útil si en redes neuronales quieres una fila o una columna de una matriz, porque al no obtener la fila o la columna haciendo slicing, es decir, `matriz[i]`, sino que se obtiene mediante una multiplicación, esta operación es deribable y por tanto se podría añadir al algoritmo del descenso del gradiente

## Word embedding

El one-hot encoding está muy bien, pero crea un nuevo problema, y es que en un lenguaje de un millón de palabras, necesitaríamos que cada palabra se codificase con vectores de 999.999 ceros y 1 solo uno. Esto a la hora de hacer operaciones matriciales no es muy eficiente y hace que las redes y datos ocupen mucha memoria

Para solucionar esto se empezó a usar el hot encoding a secas o también llamado word embedding, que consiste en seguir representando cada palabra en vectores, pero ahora cada vector tendrá un tamaño fijo, de momento digamos N, y en el que cada uno de sus componentes puede tener cualquier valor.

Esto resuelve el problema de la memoria y la ineficiencia en las operaciones matriciales

Además crea una nueva característica, ya que al representar las palabras en vectores de N dimensiones, en realidad lo que estamos haciendo es representar las palabras en un espacio N dimensional. Si esto te suena a muy complicado no te preocupes, que ahora te lo cuento de otra manera , ya verás que sencillo es y cómo lo vas a entender

Supongamos que en vez de ser un espacio de N dimensiones, tenemos un espacio de 2 dimensiones, alto y ancho, pues podríamos representar las palabras de esta manera

![word embedding 2 dimmension](Imagenes/word_embedding_2_dimmension.png)

Como puedes ver todas las palabras que tienen semejanza están juntas. PUes ahora supón que en vez de 2 dimensiones tenemos 3, alto, ancho y profundo, podríamos representar las palabras de esta manera

![word embedding 3 dimmension](Imagenes/word_embedding_3_dimmension.png)

Ahora las palabras siguen estando juntas por semejanza, pero tenemos una dimensión más para poder hacer más grupos.

Como en el lenguaje tenemos muchisimas palabras no nos vale con 3 dimensiones, por lo que tenemos que hacerlo con muchas más. El modelo más grande de word embeding de Bert es de 1024 dimensiones, mientras que el modelo más grande de word embeding de GPT3 es de 4096 dimensiones

Esta posibilidad de poder representar las palabras en grupos por semejanza también nos da otra ventaja, y es la relación entre palabras. Si al vector que representa la palabra `rey` le restas el vector que representa la palabra `hombre` y le sumas el vector que representa la palabra `mujer` obtienes un vector muy parecido al que representa la palabra `reina`.

Como hemos dicho, en el lenguaje existe una gran relación entre las palabras, lo cual va a ser muy importante para entender frases y además es uno de los mecanismos más importantes de los transformers, el de atención, de hecho el paper de los transformers se llama `Attention is all you need` (`Atención es todo lo que necesitas`). Así que gracias a esta forma de representar las palabras podemos prestar atención a esta relación entre palabras